In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

Select 20 categoies

In [3]:
import os
import shutil

# Source and destination paths
source_path = '/Users/ruoqiyan/Desktop/690ML/ml_project/data/images/'
# Get a list of all subdirectories (categories)
categories = sorted([d for d in os.listdir(source_path) if os.path.isdir(os.path.join(source_path, d))])

print(f"Total Categories: {len(categories)}")
print("Categories:")
print(categories)

Total Categories: 101
Categories:
['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito', 'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake', 'ceviche', 'cheese_plate', 'cheesecake', 'chicken_curry', 'chicken_quesadilla', 'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder', 'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes', 'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict', 'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras', 'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari', 'fried_rice', 'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad', 'grilled_cheese_sandwich', 'grilled_salmon', 'guacamole', 'gyoza', 'hamburger', 'hot_and_sour_soup', 'hot_dog', 'huevos_rancheros', 'hummus', 'ice_cream', 'lasagna', 'lobster_bisque', 'lobster_roll_sandwich', 'macaroni_

In [6]:
data_path = '/Users/ruoqiyan/Desktop/690ML/ml_project/food_20_data/images/'

# select 20 categories
selected_categories = categories[0:20]

# Create destination directory if not exists
os.makedirs(data_path, exist_ok=True)

# Copy selected categories to the new dataset folder
for category in selected_categories:
    src = os.path.join(source_path, category)
    dst = os.path.join(data_path, category)
    if os.path.exists(src):
        shutil.copytree(src, dst)
print("Dataset reduced to 20 categories!")

FileExistsError: [Errno 17] File exists: '/Users/ruoqiyan/Desktop/690ML/ml_project/food_20_data/images/apple_pie'

Model

In [8]:
# Dataset directory
dataset_path = data_path

# Image dimensions (299x299 for InceptionV3 or 224x224 for ResNet50)
img_size = (299, 299)
batch_size = 32

# Data Augmentation and Data Generators
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Split 20% for validation
)

# Load training data
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

# Load validation data
validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)



# Load the InceptionV3 model with ImageNet weights, excluding the top layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(*img_size, 3))

# Freeze all layers in the base model to retain pre-trained features
for layer in base_model.layers:
    layer.trainable = False

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(20, activation='softmax')(x)  # 20 categories for the output layer

# Final model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model with categorical cross-entropy loss
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Model summary
model.summary()


from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Set up early stopping and model checkpoint
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_food20_model.h5', save_best_only=True)
]

# Train the model
history = model.fit(
    train_generator,
    epochs=20,  # Adjust based on validation performance
    validation_data=validation_generator,
    callbacks=callbacks
)




Found 16000 images belonging to 20 classes.
Found 4000 images belonging to 20 classes.


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d_188 (Conv2D)         (None, 149, 149, 32)         864       ['input_3[0][0]']             
                                                                                                  
 batch_normalization_188 (B  (None, 149, 149, 32)         96        ['conv2d_188[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_188 (Activation  (None, 149, 149, 32)         0         ['batch_normalization_18

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/20
500/500 [==============================] - 544s 1s/step - loss: 1.5227 - accuracy: 0.5428 - val_loss: 1.2750 - val_accuracy: 0.6217
Epoch 3/20
500/500 [==============================] - 547s 1s/step - loss: 1.3662 - accuracy: 0.5846 - val_loss: 1.2021 - val_accuracy: 0.6405
Epoch 4/20
500/500 [==============================] - ETA: 0s - loss: 1.2866 - accuracy: 0.6069

KeyboardInterrupt: 

Evaluation

In [ ]:
# Unfreeze the last 30 layers of the base model
for layer in base_model.layers[-30:]:
    layer.trainable = True

# Re-compile with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
history_finetune = model.fit(
    train_generator,
    epochs=10,  # Adjust as needed
    validation_data=validation_generator,
    callbacks=callbacks
)



In [ ]:
# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")


Prediction